# Import Library

In [1]:
import datetime
from tqdm import tqdm

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LinearRegression

# Selling Data

In [2]:
selling=pd.read_csv('./data/selling.csv')
selling

,id,timestamp,item,co_op,loc,supply,price,year,month,week,day,yemon,holiday
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.00,0.00,2019,1,1,1,2019-01,1
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.00,0.00,2019,1,1,2,2019-01,0
2,TG_A_J_20190103,2019-01-03,TG,A,J,606.01,17.28,2019,1,1,3,2019-01,0
3,TG_A_J_20190104,2019-01-04,TG,A,J,250.00,14.08,2019,1,1,4,2019-01,0
4,TG_A_J_20190105,2019-01-05,TG,A,J,323.52,12.50,2019,1,1,5,2019-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,4524.40,4.68,2023,2,9,0,2023-02,0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,4219.80,5.31,2023,2,9,1,2023-02,0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,3829.80,5.74,2023,3,9,2,2023-03,0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,4772.20,5.23,2023,3,9,3,2023-03,0


## Some Basic Statistics

In [3]:
selling.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59397 entries, 0 to 59396
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         59397 non-null  object 
 1   timestamp  59397 non-null  object 
 2   item       59397 non-null  object 
 3   co_op      59397 non-null  object 
 4   loc        59397 non-null  object 
 5   supply     59397 non-null  float64
 6   price      59397 non-null  float64
 7   year       59397 non-null  int64  
 8   month      59397 non-null  int64  
 9   week       59397 non-null  int64  
 10  day        59397 non-null  int64  
 11  yemon      59397 non-null  object 
 12  holiday    59397 non-null  int64  
dtypes: float64(2), int64(5), object(6)
memory usage: 5.9+ MB


In [4]:
selling.describe()

,supply,price,year,month,week,day,holiday
count,59397.000000,59397.000000,59397.000000,59397.000000,59397.000000,59397.000000,59397.00000
mean,118.945248,11.316807,2020.601445,6.320420,25.772160,2.998687,0.03985
std,522.639995,20.299414,1.201092,3.519942,15.382153,1.998374,0.19561
min,0.000000,0.000000,2019.000000,1.000000,1.000000,0.000000,0.00000
25%,0.000000,0.000000,2020.000000,3.000000,12.000000,1.000000,0.00000
50%,0.000000,0.000000,2021.000000,6.000000,26.000000,3.000000,0.00000
75%,38.000000,15.190000,2022.000000,9.000000,39.000000,5.000000,0.00000
max,12228.000000,209.090000,2023.000000,12.000000,53.000000,6.000000,1.00000


In [ ]:
selling.corr(numeric_only=True)

## Draw Boxplot & Lineplot about 'supply' & 'price' column

In [ ]:
model = LinearRegression()
model.fit(X=selling[selling['supply']>0]['supply'].to_numpy().reshape(-1,1),
          y=selling[selling['supply']>0]['price'])
pred = model.predict(selling[selling['supply']>0]['supply'].to_numpy().reshape(-1,1))

fig = plt.figure(figsize=(10, 6))
sns.scatterplot(data=selling[selling['supply']>0],
            x = "supply",
            y = "price")
sns.lineplot(data=selling[selling['supply']>0],
            x="supply",
            y=pred,
            color='orange')
plt.xlabel('supply(100kg)')
plt.ylabel('price (won/100kg)')
plt.show()

In [ ]:
fig = plt.figure(figsize=(20, 8))
sns.boxplot(data=selling,
            x = "item",
            y = "supply")
plt.ylabel('supply (100kg)')
plt.show() # 특이사항 없음

In [ ]:
fig = plt.figure(figsize=(20, 8))
sns.lineplot(data=selling,
             x='yemon',
             y='supply',
             hue='item',
            ci=None)
plt.xticks(rotation=45)
plt.xlabel('timestamp')
plt.ylabel('supply (100kg)')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 6))
sns.boxenplot(data=selling[selling['price']>0],
            x = "item",
            y = "price")
plt.ylabel('price (won/100kg)')
plt.show() # 감귤(TG) 혼자 상이한 분포를 가진 모습

In [ ]:
fig = plt.figure(figsize=(20, 8))
sns.lineplot(data=selling,
             x='yemon',
             y='price',
             hue='item',
             ci=None)
plt.xticks(rotation=45)
# ax = plt.gca()
# ax.tick_params(labelbottom=False)
plt.xlabel('timestamp')
plt.ylabel('price (won/100kg)')
plt.show()

In [ ]:
df=selling[(selling['item']=='TG')&(selling['price']>0)].reset_index(drop=True)
sns.histplot(data=df,
             x='price',
             binwidth=2,
             kde=True
            )
plt.xticks(rotation=45)
plt.xlabel('price (won/100kg)')
plt.show() # 두 개의 가우시안 분포가 혼합되어 있는 모습 발견

## Draw Histplot about Each Items

In [ ]:
df=selling[(selling['item']=='CR')&(selling['price']>0)].reset_index(drop=True)
sns.histplot(data=df,
             x='price',
             binwidth=2,
            color='orange',
             kde=True
            )
plt.xticks(rotation=45)
plt.xlabel('price (won/100kg)')
plt.show()

In [ ]:
df=selling[(selling['item']=='CB')&(selling['price']>0)].reset_index(drop=True)
sns.histplot(data=df,
             x='price',
             binwidth=2,
            color='green',
             kde=True
            )
plt.xticks(rotation=45)
plt.xlabel('price (won/100kg)')
plt.show()

In [ ]:
df=selling[(selling['item']=='RD')&(selling['price']>0)].reset_index(drop=True)
sns.histplot(data=df,
             x='price',
             binwidth=2,
             color='red',
             kde=True
            )
plt.xticks(rotation=45)
plt.xlabel('price (won/100kg)')
plt.show()

In [ ]:
df=selling[(selling['item']=='BC')&(selling['price']>0)].reset_index(drop=True)
sns.histplot(data=df,
             x='price',
             binwidth=2,
             color='purple',
             kde=True
            )
plt.xticks(rotation=45)
plt.xlabel('price (won/100kg)')
plt.show()

In [ ]:
fig, axes = plt.subplots(5, 1, figsize=(10, 8))

df=selling[(selling['item']=='BC')&(selling['price']>0)].reset_index(drop=True)
sns.histplot(data=df,
             x='price',
             binwidth=2,
             color='purple',
             kde=True,
             ax=axes[4]
            )
axes[4].set_title('BC',fontsize=8)
axes[4].tick_params(labelsize=8)

df=selling[(selling['item']=='CB')&(selling['price']>0)].reset_index(drop=True)
sns.histplot(data=df,
             x='price',
             binwidth=2,
            color='green',
             kde=True,
             ax=axes[2]
            )
axes[2].set_title('CB',fontsize=8)
axes[2].tick_params(labelsize=8)

df=selling[(selling['item']=='RD')&(selling['price']>0)].reset_index(drop=True)
sns.histplot(data=df,
             x='price',
             binwidth=2,
             color='red',
             kde=True,
             ax=axes[3]
            )
axes[3].set_title('RD',fontsize=8)
axes[3].tick_params(labelsize=8)

df=selling[(selling['item']=='CR')&(selling['price']>0)].reset_index(drop=True)
sns.histplot(data=df,
             x='price',
             binwidth=2,
            color='orange',
             kde=True,
             ax=axes[1]
            )
axes[1].set_title('CR',fontsize=8)
axes[1].tick_params(labelsize=8)

df=selling[(selling['item']=='TG')&(selling['price']>0)].reset_index(drop=True)
sns.histplot(data=df,
             x='price',
             binwidth=2,
             kde=True,
             ax=axes[0]
            )
axes[0].set_title('TG',fontsize=8)
axes[0].tick_params(labelsize=8)

plt.xlabel('price (won/100kg)')
plt.ylabel('count')

plt.show()

## Devide TG with Gaussian Mixture Model

In [ ]:
# GMM 모델 사용을 위해 이상치 파악 -> 해당 이상치 인덱스 백업 후 제거
tg_df=selling[(selling['item']=='TG')&(selling['price']>0)]
sns.boxplot(data=tg_df,
            x = "item",
            y = "price")
plt.ylabel('price (won/100kg)')
plt.show()

In [ ]:
Q1 = tg_df[['price']].quantile(q=0.25).iloc[0]
Q3 = tg_df[['price']].quantile(q=0.75).iloc[0]
IQR = Q3-Q1

out_idx=[]
for idx in range(len(tg_df)):
    if tg_df.iloc[idx,6] > Q3+1.5*IQR:
        out_idx.append(tg_df.index[idx])

backup=tg_df.loc[out_idx,:]
backup['expen']='expensive'

tg_df=tg_df.drop(out_idx,axis=0)
tg_df

### Fit and Predict

In [ ]:
tg_dff=tg_df[['price']]
gmm = GaussianMixture(n_components=2, random_state=42, n_init=10, max_iter=300).fit(tg_dff)
gmm_cluster_labels = gmm.predict(tg_dff)
tg_df['expen'] = gmm_cluster_labels
tg_df['expen'] = np.where(tg_df['expen'] == 0, 'expensive', 'cheap')
tg_df

### Scatterplot & Histplot

In [ ]:
sns.scatterplot(data=tg_df,
                x='month',
                y='price',
                hue='expen',
                palette={'cheap':'navy',
                   'expensive':'gold'}
            )
plt.xticks(rotation=45)
plt.ylabel('price (won/100kg)')
plt.show()

In [ ]:
fig = plt.figure(figsize=(20, 8))
sns.histplot(data=tg_df,
             x='yemon',
             hue='expen',
             alpha=0.05,
             palette={'cheap':'navy',
                   'expensive':'gold'},
             edgecolor='0.95',
             kde=True
            )

plt.xticks(rotation=45)
plt.xlabel('timestamp')
plt.show()

In [ ]:
fig = plt.figure(figsize=(20, 8))
sns.histplot(data=tg_df,
             x='month',
             hue='expen',
            binrange=(0.5,12.5),
            binwidth=1,
             palette={'cheap':'navy',
                   'expensive':'gold'}
            )

plt.show()

# 3월부터 9월까지는 비싼 감귤 (만감류&하우스)
# 10월부터 2월까지는 싼 감귤 (노지) 이 주로 판매됨

In [ ]:
# 백업해뒀던 데이터 프레임 붙여넣고 sort
tg_df=pd.concat([tg_df,backup]).sort_index()
tg_df

### TG -> C(heap)TG/E(xpensive)TG in Selling Table

In [ ]:
for idx in range(len(tg_df)):
    if tg_df.iloc[idx,-1]=='expensive':
        tg_df.iloc[idx,2]='ETG'
    else:
        tg_df.iloc[idx,2]='CTG'
tg_df

In [ ]:
selling['item'].value_counts()

In [ ]:
for idx in tg_df.index:
    selling.loc[idx,'item']=tg_df.loc[idx,'item']
    
# TG가 사라질 때까지 다음 행의 CTG, ETG 중 하나로 덮어쓰기
for _ in tqdm(range(selling['item'].value_counts()['TG'])):
    for idx in selling[selling['item']=='TG'].index:
        if selling.loc[idx,'item']=='TG':
            selling.loc[idx,'item']=selling.loc[idx+1,'item']
        if 'TG' not in selling['item'].unique():
            break
selling

In [ ]:
# 전체 건수 대비 공휴일 건수 비율을 비교해 holiday 컬럼에 대한 타당성 확인
all_item=selling['item'].value_counts()
holi_item=selling[selling['holiday']==1]['item'].value_counts()
all_holi=pd.DataFrame([all_item,holi_item]).T
all_holi.columns=['all_cnt','holi_cnt']
all_holi['ratio']=round(all_holi['holi_cnt']/all_holi['all_cnt'],4)
all_holi

In [ ]:
fig = plt.figure(figsize=(20, 8))
sns.boxplot(data=selling[selling['price']>0],
            x = "item",
            y = "price",
           palette={'CTG':'navy',
                      'ETG':'gold',
                      'CR':'orange',
                      'CB':'green',
                      'RD':'red',
                      'BC':'purple'}
           )
plt.ylabel('price')
plt.show() # 비싼 감귤(ETG) 혼자 상이한 분포를 가진 모습

In [ ]:
fig = plt.figure(figsize=(20, 8))
sns.lineplot(data=selling[(selling['item']=='CTG')|(selling['item']=='ETG')],
             x='yemon',
             y='price',
             hue='item',
             palette={'CTG':'navy',
                      'ETG':'gold'
#                       'CR':'orange',
#                       'CB':'green',
#                       'RD':'red',
#                       'BC':'purple'
                     },
             ci=None)
plt.xticks(rotation=45)
plt.xlabel('timestamp')
plt.ylabel('price (won/100kg)')
plt.show()

In [ ]:
selling

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10, 8))

df=selling[(selling['item']=='CTG')&(selling['price']>0)].reset_index(drop=True)
sns.histplot(data=df,
             x='price',
             binwidth=2,
             color='gold',
             kde=True,
             ax=axes[0]
            )
axes[0].set_title('CTG',fontsize=8)
axes[0].tick_params(labelsize=8)

df=selling[(selling['item']=='ETG')&(selling['price']>0)].reset_index(drop=True)
sns.histplot(data=df,
             x='price',
             binwidth=2,
            color='navy',
             kde=True,
             ax=axes[1]
            )
axes[1].set_title('ETG',fontsize=8)
axes[1].tick_params(labelsize=8)

In [ ]:
# selling.to_csv('./data/gmm_selling.csv',index=False)

# With ASOS Data

## Load Data

In [ ]:
gmm_selling=pd.read_csv('./data/gmm_selling.csv')
gmm_selling

In [ ]:
asos = pd.read_csv('./data/asos.csv')
asos

## Sort Selling Data

In [ ]:
gmm_selling=gmm_selling.sort_values(['loc','item','timestamp']).reset_index(drop=True)
gmm_selling

## Add Columns about Weather Data

In [ ]:
gmm_selling['avg_tem']=0
gmm_selling['max_tem']=0
gmm_selling['min_tem']=0
# gmm_selling['sum_rain']=0
gmm_selling['max_rain']=0
gmm_selling['hr_max_rain']=0
gmm_selling['max_wind']=0
gmm_selling['avg_sun']=0
agri_date={'BC':['07','10'],
           'CR':['08','11'],
           'CB':['08','11'],
           'RD':['09','11'],
           'CTG':['03','10'],
           'ETG':['10','02']}
item_uniq=gmm_selling['item'].unique()
year_unqi=list(map(lambda x:x-1,list(selling['year'].unique())))
locate=['J','S']
for loc in locate:
    for item in item_uniq:
        agri_loc=asos[asos['station']==loc].reset_index(drop=True)
        print(f'{loc} - {item}')
        for y in year_unqi:
            if int(agri_date[item][0])>int(agri_date[item][1]):
                agri_weather=agri_loc[(agri_loc['timestamp']>=(str(y-1)+'-'+str(agri_date[item][0])))&(agri_loc['timestamp']<=(str(y)+'-'+str(agri_date[item][1])))]
            else:
                agri_weather=agri_loc[(agri_loc['timestamp']>=(str(y)+'-'+str(agri_date[item][0])))&(agri_loc['timestamp']<=(str(y)+'-'+str(agri_date[item][1])))]
            avg_tem=round(agri_weather['temperture'].mean(),3)
            max_tem=max(agri_weather['maxtem'])
            min_tem=min(agri_weather['mintem'])
#             sum_rain=sum(agri_weather['raining'])
            max_rain=max(agri_weather['raining'])
            hr_max_rain=max(agri_weather['hrrain'])
            max_wind=max(agri_weather['wind_speed'])
            avg_sun=round(agri_weather['sunlight_hour'].mean(),3)
            
            idx_list=gmm_selling[(gmm_selling['item']==item)&(gmm_selling['loc']==loc)&(gmm_selling['timestamp']>=(str(y)+'-'+str(agri_date[item][1])))&(gmm_selling['timestamp']<=(str(y+1)+'-'+str(agri_date[item][1])))].index
            for idx in tqdm(idx_list):
                gmm_selling.loc[idx,'avg_tem']=avg_tem
                gmm_selling.loc[idx,'max_tem']=max_tem
                gmm_selling.loc[idx,'min_tem']=min_tem
#                 gmm_selling.loc[idx,'sum_rain']=sum_rain
                gmm_selling.loc[idx,'max_rain']=max_rain
                gmm_selling.loc[idx,'hr_max_rain']=hr_max_rain
                gmm_selling.loc[idx,'max_wind']=max_wind
                gmm_selling.loc[idx,'avg_sun']=avg_sun
        

In [ ]:
gmm_selling

In [ ]:
# gmm_selling.to_csv('./data/asos_selling.csv',index=False)

## Check Correlation between 'price' & weather columns

In [ ]:
asos_selling=pd.read_csv('./data/asos_selling.csv')
asos_selling=asos_selling[asos_selling['price']>0].reset_index(drop=True)
asos_selling

In [ ]:
aso=asos_selling.copy()
aso=aso[['item','supply','price','avg_tem','max_tem','min_tem',
         'max_rain','hr_max_rain','max_wind','avg_sun']]
corr = aso.corr(numeric_only=True)
corr

In [ ]:
fig, ax = plt.subplots( figsize=(14,14) )

aso_copy=aso.drop(['item'],axis=1)

mask = np.zeros_like(aso_copy.corr(numeric_only=True), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

test_heatmap = sns.heatmap(corr.values,
                          cbar = True,
                           cbar_kws={"shrink": .5},
                           cmap = 'RdYlBu_r',
                           annot = True,
                          annot_kws={'size' : 20},
                           mask=mask,
                           fmt = '.2f',
                           square = 'True',
                           linewidths=.5,
                          yticklabels=aso_copy.columns,
                          xticklabels=aso_copy.columns,
                          vmin = -1,vmax = 1)
plt.show()

In [ ]:
BC_sell=aso[aso['item']=='BC'].reset_index(drop=True)
BC_sell.corr(numeric_only=True)

In [ ]:
CB_sell=aso[aso['item']=='CB'].reset_index(drop=True)
CB_corr=CB_sell.corr(numeric_only=True)
CB_corr

In [ ]:
fig, ax = plt.subplots( figsize=(14,14) )
CB_sell=CB_sell.drop(['item'],axis=1)

mask = np.zeros_like(CB_sell.corr(numeric_only=True), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

test_heatmap = sns.heatmap(CB_corr.values,
                          cbar = True,
                           cbar_kws={"shrink": .5},
                           cmap = 'RdYlBu_r',
                           annot = True,
                          annot_kws={'size' : 20},
                           mask=mask,
                           fmt = '.2f',
                           square = 'True',
                           linewidths=.5,
                          yticklabels=CB_sell.columns,
                          xticklabels=CB_sell.columns,
                          vmin = -1,vmax = 1)
plt.show()

In [ ]:
CR_sell=aso[aso['item']=='CR'].reset_index(drop=True)
CR_sell.corr(numeric_only=True)

In [ ]:
RD_sell=aso[aso['item']=='RD'].reset_index(drop=True)
RD_sell.corr(numeric_only=True)

In [ ]:
CTG_sell=aso[aso['item']=='CTG'].reset_index(drop=True)
CTG_sell.corr(numeric_only=True)

In [ ]:
ETG_sell=aso[aso['item']=='ETG'].reset_index(drop=True)
ETG_sell.corr(numeric_only=True)

In [ ]:
fig = plt.figure(figsize=(20, 8))
sns.lineplot(data=asos_selling,
             x='yemon',
             y='avg_tem',
             ci=None,
            label='Average Temperature')
sns.lineplot(data=asos_selling,
             x='yemon',
             y='price',
             ci=None,
            label='Price')
plt.xticks(rotation=45)
plt.xlabel('timestamp')
plt.ylabel('')
plt.legend()
plt.title('Price & Average Temperature Trends')
plt.show()

In [ ]:
CB=CB_sell.groupby('max_rain',as_index=False)['price'].mean()
model = LinearRegression()
model.fit(X=CB['max_rain'].to_numpy().reshape(-1,1),
          y=CB['price'])
pred = model.predict(CB['max_rain'].to_numpy().reshape(-1,1))
pred

sns.scatterplot(data=CB,
                x='max_rain',
                y='price'
            )
sns.lineplot(x=CB['max_rain'].to_numpy(),
            y=pred,
            color='orange')
# plt.xticks(rotation=45)
# plt.ylabel('price (won/100kg)')
plt.title('Scatter Max Raining between Price (Cabbage)')
plt.show()

In [ ]:
CB=CB_sell.groupby('avg_sun',as_index=False)['price'].mean()
model = LinearRegression()
model.fit(X=CB['avg_sun'].to_numpy().reshape(-1,1),
          y=CB['price'])
pred = model.predict(CB['avg_sun'].to_numpy().reshape(-1,1))
pred

sns.scatterplot(data=CB,
                x='avg_sun',
                y='price'
            )
sns.lineplot(x=CB['avg_sun'].to_numpy(),
            y=pred,
            color='green')
# plt.xticks(rotation=45)
# plt.ylabel('price (won/100kg)')
plt.title('Scatter Average Sunlight between Price (Cabbage)')
plt.show()